In [1]:
import sys
sys.path.append('../scripts')
from course_utils import trainTest
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from models import evaluate_accuracy
import models
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
import pickler

##TODO: 
- Run on larger data set 
- lower min-n-gram count
- try TF-IDF True or false

In [3]:
mydat = pickler.load_object('../data/data_and_labels.pkl')
X = mydat['X']
y = mydat['y']
ordered_case_ids = mydat['ordered_case_ids']

In [9]:
def subsample(X,y,case_ids, sample_pct):
    print sample_pct
    
np.randint(4)

AttributeError: 'module' object has no attribute 'randint'

In [5]:
def train_test_split(X,y, ordered_case_ids,pct_train):
    train_rows = int(pct_train*len(y))
    y_train = np.array(y[:train_rows])
    y_test = np.array(y[train_rows:])
    X_train = X[:train_rows]
    X_test = X[train_rows:]
    case_ids_train = ordered_case_ids[:train_rows]
    case_ids_test = ordered_case_ids[train_rows:]
    return X_train,y_train,case_ids_train,X_test,y_test,case_ids_test

X_train,y_train,case_ids_train,X_test,y_test,case_ids_test = train_test_split(X,y,ordered_case_ids,0.75)

In [6]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(11599, 351415)
(3867, 351415)
(11599,)
(3867,)


##Baseline Model
The base line majority classifier model to beat is 54% accuracy

In [13]:
reload(models)
mclf = models.MajorityClassifier()
mclf.fit(X_train,y_train)

evaluate_accuracy(y_test,mclf.predict(X_test))

	 	 pred
true 	 	 1 	 2 	 3
	 1 	 2154 	 0 	 0
	 2 	 627 	 0 	 0
	 3 	 1086 	 0 	 0


0.55702094647013189

##Out-of-the-Box SVM
Gets accuracy of 39%

In [14]:
clf = LinearSVC(penalty='l1',random_state=0, dual=False)
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=0, tol=0.0001,
     verbose=0)

In [15]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [16]:
clf.score(X_test,y_test)

0.57124385828807867

In [17]:
evaluate_accuracy(y_train,y_train_pred)

	 	 pred
true 	 	 1 	 2 	 3
	 1 	 5013 	 87 	 551
	 2 	 364 	 1213 	 263
	 3 	 797 	 81 	 3230


0.81524269333563237

In [12]:
evaluate_accuracy(y_test,y_test_pred)

	 	 pred
true 	 	 1 	 2 	 3
	 1 	 1736 	 69 	 349
	 2 	 382 	 98 	 147
	 3 	 648 	 63 	 375


0.57124385828807867

##Optimized SVM
Accuracy is truly horrendous: 40%

In [7]:
def optimizeSVM(X_train, y_train, reg_min_log10=-2, reg_max_log10=2, regularization_type='l1'):
    '''
    Creates an SVM classifier trained on the given data with an optimized C parameter.
    Args:
      X_train: A dataframe on which to train the features
      y_train: A dataframe on which to evaluate the training data
      reg_min_log10: log base 10 of the low end of the regularization parameter range.  -2 means 10^-2
      reg_max_log10: log base 10 of the high end of the regularization parameter range.  2 means 10^2
    Returns:
      A fitted SVM classifier.
    '''
    
    model_to_set = LinearSVC(penalty=regularization_type,random_state=0, dual=False)
    # consider broadening the param_grid to include different SVM kernels and degrees.  See:
    # http://stackoverflow.com/questions/12632992/gridsearch-for-an-estimator-inside-a-onevsrestclassifier
    #param_grid = {'C': [10**i for i in range(-reg_min_log10,reg_max_log10)] + [1e30]}
    param_grid = {'C':[1]}
    model_tuning = GridSearchCV(model_to_set, scoring='f1_weighted',param_grid=param_grid)
    
    model_tuning.fit(X_train, y_train)
    print 'best C param for SVM classifier:', model_tuning.best_params_['C']
    print 'best_score: ', model_tuning.best_score_
        
    return model_tuning.best_estimator_

In [ ]:
svm_opt = optimizeSVM(X_train,y_train,-1,1)

In [ ]:
svm_opt.score(X_test,y_test)

In [ ]:
evaluate_accuracy(svm_opt.predict(X_test),y_test)

In [ ]:
def optimizeLogistic(X_train, y_train, reg_min_log10=-2, reg_max_log10=2,regularization_type='l1'):
    '''
    Creates an SVM classifier trained on the given data with an optimized C parameter.
    Args:
      X_train: A dataframe on which to train the features
      y_train: A dataframe on which to evaluate the training data
      score_func: Scoring function.  Dizzying options here.  Consider:
          metrics.accuracy_score
          metrics.f1_score
    Returns:
      A fitted SVM classifier.
    '''
    
    model_to_set = LogisticRegression(penalty=regularization_type)
    param_grid = {'C': [10**i for i in range(-reg_min_log10,reg_max_log10)] + [1e30]}
    model_tuning = GridSearchCV(model_to_set, param_grid=param_grid,
                             scoring='f1_weighted')
    
    model_tuning.fit(X_train, y_train)
    print 'best C param for LR classifier:', model_tuning.best_params_['C']
    print 'best params: ', model_tuning.best_params_
    print 'best_score: ', model_tuning.best_score_
        
    return model_tuning.best_estimator_

In [ ]:
logit_opt = optimizeLogistic(X_train,y_train)

In [ ]:
logit_opt.score(X_test,y_test)

In [ ]:
evaluate_accuracy(logit_opt.predict(X_test),y_test)